# ein Python programm zur Analyse eine .har datei


Es soll eine Webseite analyiert werden, in der eine har datei geprüft wird

In [ ]:
import json
from haralyzer import HarParser
import pandas as pd
import matplotlib.pyplot as plt

# Laden der HAR-Datei
with open('example.har', 'r') as f:
    har_data = json.load(f)

# Initialisieren des HarParser
har_parser = HarParser(har_data)

# Erstellen einer Liste für die Daten und URLs
data = []
urls = []

In [ ]:
# Durchlaufen aller Seiten in der HAR-Datei
for page in har_parser.pages:
    for entry in page.entries:
        url = entry['request']['url']
        domain = url.split('/')[2]
        bytes_transferred = entry['response']['bodySize']
        
        # Hinzufügen der Daten zur Liste
        data.append({'domain': domain, 'bytes': bytes_transferred})
        # Hinzufügen der URL zur Liste
        urls.append(url)




In [ ]:

# Erstellen eines DataFrames aus der Liste
df = pd.DataFrame(data)

# Gruppieren nach Domain und Summieren der übertragenen Bytes
domain_data = df.groupby('domain').sum().reset_index()

# Erstellen des Kuchendiagramms
plt.figure(figsize=(10, 7))
plt.pie(domain_data['bytes'], labels=domain_data['domain'], autopct='%1.1f%%', startangle=140)
plt.axis('equal')  # Gleichmäßiges Kreisdiagramm
plt.title('Datenverteilung nach Domain')
plt.show()

print("HAR-Datei und URL-Liste wurden erfolgreich erstellt.")

In [ ]:
print (urls)
for url in urls:
    print (url)
